## MLP3

#### Load data and training-test split

#### Model 1: Decision tree – information gain (entropy)

#### Model 2: kNN classifier

#### Model 3: Naive Bayes classifier or logistic regression??? idk